In [1]:
# Import whatever will be used in this notebook
import pylab
import math
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt


In [2]:
#read in the csv file and remove the extra ID column
credit = pd.read_csv("C:\\Users\\Jeroen\\Desktop\\Ubiqum\\Data Science\\Excel Files\\credit_1.csv", header = 0)
credit = credit[credit.columns[1:len(credit.columns)]]

#Give head of the df so it is easily visible which vars should function as independent variable
credit.head()

,ID,CREDIT,SEX,EDUCATION,MARRIAGE,AGE,PAYSTAT_APR,PAYSTAT_MAY,PAYSTAT_JUNE,PAYSTAT_JULY,...,CUR_BIL_JULY,CUR_BIL_AUG,CUR_BIL_SEP,PAID_APR,PAID_MAY,PAID_JUNE,PAID_JULY,PAID_AUG,PAID_SEP,DEFAULT
0,1,20000,Female,University,Single,24,-2,-2,-1,-1,...,689,3102,3913,0,0,0,0,689,0,Defaults
1,2,120000,Female,University,Married,26,2,0,0,0,...,2682,1725,2682,2000,0,1000,1000,1000,0,Defaults
2,3,90000,Female,University,Married,34,0,0,0,0,...,13559,14027,29239,5000,1000,1000,1000,1500,1518,Pays
3,4,50000,Female,University,Single,37,0,0,0,0,...,49291,48233,46990,1000,1069,1100,1200,2019,2000,Pays
4,5,50000,Male,University,Single,57,0,0,0,-1,...,35835,5670,8617,679,689,9000,10000,36681,2000,Pays


In [3]:
#Generate dummy arrays for each month 
dummies_apr = pd.get_dummies(credit['PAYSTAT_APR'])
dummies_may = pd.get_dummies(credit['PAYSTAT_MAY'])
dummies_june = pd.get_dummies(credit['PAYSTAT_JUNE'])
dummies_july = pd.get_dummies(credit['PAYSTAT_JULY'])
dummies_aug = pd.get_dummies(credit['PAYSTAT_AUG'])
dummies_sep = pd.get_dummies(credit['PAYSTAT_SEP'])    

In [4]:
#combine april and may since they dont have values of 1 
dummies_aprmay = dummies_apr + dummies_may
#combine june, july, august, and september 
dummies_summer = dummies_june + dummies_july + dummies_aug + dummies_sep

In [5]:
#add the 2 dataframes together
dummies_total = dummies_aprmay + dummies_summer
del dummies_total[1]
dummies_total.insert(3, 1, dummies_summer[3])

In [6]:
# Change the names of the columns to what they represent
dummies_total.columns = ['No_consumption', 'Paid_in_full', 'Paid_partly', 'Delay_1_mo', 'Delay_2_mo', 'Delay_3_mo', 'Delay_4_mo', 'Delay_5_mo', 'Delay_6_mo', 'Delay_7_mo', 'Delay_8_mo']
dummies_total.head()

,No_consumption,Paid_in_full,Paid_partly,Delay_1_mo,Delay_2_mo,Delay_3_mo,Delay_4_mo,Delay_5_mo,Delay_6_mo,Delay_7_mo,Delay_8_mo
0,2,2,0,0,2,0,0,0,0,0,0
1,0,1,3,0,2,0,0,0,0,0,0
2,0,0,6,0,0,0,0,0,0,0,0
3,0,0,6,0,0,0,0,0,0,0,0
4,0,2,4,0,0,0,0,0,0,0,0


In [7]:
#Sum the columns with a more than 3 month delay in a new array
Delay_more_Mo = (dummies_total['Delay_4_mo'] + dummies_total['Delay_5_mo'] + dummies_total['Delay_6_mo'] + 
                 dummies_total['Delay_7_mo'] + dummies_total['Delay_8_mo'])

#Remove the columns with more than 3 months delay from the total set 
del dummies_total['Delay_4_mo']
del dummies_total['Delay_5_mo']
del dummies_total['Delay_6_mo']
del dummies_total['Delay_7_mo']
del dummies_total['Delay_8_mo']

#Combine the new array to the set
dummies_total = pd.concat([dummies_total, Delay_more_Mo], axis=1, sort=False)

#Rename the column
dummies_total = dummies_total.rename(columns = {0:'Delay_3+_mo'})

In [8]:
#Remove the specific statusses for the months and add the counted values
del credit['PAYSTAT_APR']
del credit['PAYSTAT_MAY']
del credit['PAYSTAT_JUNE']
del credit['PAYSTAT_JULY']
del credit['PAYSTAT_AUG']
del credit['PAYSTAT_SEP']

In [9]:
#add the dataframes together again 
credit_tmp = pd.concat([credit, dummies_total], axis=1, sort=False)
credit_tmp.head()

,ID,CREDIT,SEX,EDUCATION,MARRIAGE,AGE,CUR_BIL_APR,CUR_BIL_MAY,CUR_BIL_JUNE,CUR_BIL_JULY,...,PAID_AUG,PAID_SEP,DEFAULT,No_consumption,Paid_in_full,Paid_partly,Delay_1_mo,Delay_2_mo,Delay_3_mo,Delay_3+_mo
0,1,20000,Female,University,Single,24,0,0,0,689,...,689,0,Defaults,2,2,0,0,2,0,0
1,2,120000,Female,University,Married,26,3261,3455,3272,2682,...,1000,0,Defaults,0,1,3,0,2,0,0
2,3,90000,Female,University,Married,34,15549,14948,14331,13559,...,1500,1518,Pays,0,0,6,0,0,0,0
3,4,50000,Female,University,Single,37,29547,28959,28314,49291,...,2019,2000,Pays,0,0,6,0,0,0,0
4,5,50000,Male,University,Single,57,19131,19146,20940,35835,...,36681,2000,Pays,0,2,4,0,0,0,0


In [10]:
# Add dependent variable to the end of the dataset
DEFAULT = credit_tmp['DEFAULT']
del credit_tmp['DEFAULT']
del credit
credit = pd.concat([credit_tmp, DEFAULT], axis=1, sort=False)

In [11]:
credit.head()

,ID,CREDIT,SEX,EDUCATION,MARRIAGE,AGE,CUR_BIL_APR,CUR_BIL_MAY,CUR_BIL_JUNE,CUR_BIL_JULY,...,PAID_AUG,PAID_SEP,No_consumption,Paid_in_full,Paid_partly,Delay_1_mo,Delay_2_mo,Delay_3_mo,Delay_3+_mo,DEFAULT
0,1,20000,Female,University,Single,24,0,0,0,689,...,689,0,2,2,0,0,2,0,0,Defaults
1,2,120000,Female,University,Married,26,3261,3455,3272,2682,...,1000,0,0,1,3,0,2,0,0,Defaults
2,3,90000,Female,University,Married,34,15549,14948,14331,13559,...,1500,1518,0,0,6,0,0,0,0,Pays
3,4,50000,Female,University,Single,37,29547,28959,28314,49291,...,2019,2000,0,0,6,0,0,0,0,Pays
4,5,50000,Male,University,Single,57,19131,19146,20940,35835,...,36681,2000,0,2,4,0,0,0,0,Pays


In [12]:
# Save the dataframe 
credit.to_csv("C:\\Users\\Jeroen\\Desktop\\Ubiqum\\Data Science\\Excel Files\\Credit_2.csv")